In [2]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.5 MB/s eta 0:00:00


In [ ]:
# 📦 Import required libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras_tuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split

df = pd.read_csv('Real_Combine.csv')


df = df.replace([np.inf, -np.inf], np.nan)
df_clean = df.dropna()


x = df_clean.drop(['PM 2.5'], axis=1)
y = df_clean['PM 2.5']


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

def buildModel(hp):
    model = keras.Sequential()

    # Variable number of hidden layers
    for i in range(hp.Int('num_layers', 2, 5)):  # Hidden Layers From 2 to 5
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),  # No of Neurons From 32 to 512
            activation='relu'
        ))

    # Output layer (regression)
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        ),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error']
    )

    return model

tuner = RandomSearch(
    buildModel,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=2,
    directory='project1',
    project_name='HyperParameter_Tuning'
)

tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))
tuner.results_summary()

Trial 5 Complete [00h 00m 09s]
val_mean_absolute_error: 59.35737419128418

Best val_mean_absolute_error So Far: 58.51756477355957
Total elapsed time: 00h 00m 50s
